# Electricity Generation from api.electricitymap.org
---

SECTION 1 
collect data on carbon intensity, zones within SW , and power breakdown from the electricitymap api

In [386]:
# Dependencies and Setup

import pandas as pd
import requests


In [387]:

# zones for electrical utilities in US
zones = ["US-SW-PNM", "US-SW-EPE", "US-SW-WALC", "US-NW-PACE", "US-NW-PSCO", "US-CENT-SWPP", "US-TEX-ERCO", "US-MIDW-AECI","US-SW-AZPS","US-SW-AZPS",
         "US-NW-WACM", "US-SW-SRP", "US-SW-TEPC", "US-CENT-SPA", "US-CAL-IID", "US-CAL-CISO", "US-CAL-BANC","US-CAL-BANC", "US-CAL-TIDC", 
          "US-CAR-CPLE", "US-CAR-CPLW", "US-CAR-DUK", "US-CAR-SC", "US-CAR-SCEG", "US-CAR-YAD", "US-FLA-FMPP", "US-FLA-FPC" , "US-FLA-FPL",
          "US-FLA-GVL" , "US-FLA-HST", "US-FLA-JEA", "US-FLA-SEC", "US-FLA-TAL", "US-FLA-TEC", "US-MIDW-AECI" , "US-MIDW-LGEE", "US-MIDW-MISO",
          "US-NE-ISNE", "US-NW-BPAT", "US-NW-CHPD", "US-NW-DOPD", "US-NW-GCPD", "US-NW-GRID",  "US-NW-IPCO" , "US-NW-NWMT", "US-NW-NEVP", 
           "US-NW-PACW",  "US-NW-PGE", "US-NW-PSEI", "US-NW-SCL", "US-NW-TPWR", "US-NW-WAUW", "US-NY-NYIS", "US-SE-SEPA", "US-SE-SOCO" , 
           "US-TEN-TVA"]


In [388]:

# get carbon intensity history for the US utilities
urls = []
for index, url in enumerate(zones):
    url = f'https://api.electricitymap.org/v3/carbon-intensity/history?zone={zones[index]}'
    urls.append(url)

responses_dict = {}
for idx, url in enumerate(urls):
    response = requests.get(url)
    responses_dict[f"response_{idx+1}"] = response.json()

# Specify the file path where you want to save the JSON file
import json

file_path = "C_intensity_history_data.json"

# Write the dictionary to a JSON file
with open(file_path, 'w') as json_file:
    json.dump(responses_dict, json_file, indent=4)

print("Dictionary successfully exported to JSON file.")

df_carbon_intensity_history = pd.read_json(file_path)


Dictionary successfully exported to JSON file.


In [389]:
#request power breakdown
pburls = []
for index, url in enumerate(zones):
    pburl = f'https://api.electricitymap.org/v3/power-breakdown/history?zone={zones[index]}'
    pburls.append(pburl)

power_breakdown_responses_dict = {}
for idx, pburl in enumerate(pburls):
    response = requests.get(pburl)
    power_breakdown_responses_dict[f"response_{idx+1}"] = response.json()

# Specify the file path where you want to save the JSON file

file_path = "power_breakdown_history_data.json"

# Write the dictionary to a JSON file
with open(file_path, 'w') as json_file:
    json.dump(power_breakdown_responses_dict, json_file, indent=4)

df_power_breakdown_history = pd.read_json(file_path)

SECTION 2     
PowerBreakdown data transformation

In [390]:
# pull data from power breakdown json in dataframe
region = df_power_breakdown_history['response_1']['history'][0]['zone']
datetime = df_power_breakdown_history['response_1']['history'][0]['datetime']
nuclear = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['nuclear']
geothermal = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['geothermal']
biomass = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['biomass']
coal = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['coal']
wind = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['wind']
solar = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['solar']
hydro = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['hydro']
gas = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['gas']
oil = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['oil']
unknown = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['unknown']
hydro_discharge = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['hydro discharge']
battery_discharge = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['battery discharge']
renewable_percentage = df_power_breakdown_history['response_1']['history'][0]["renewablePercentage"]
total_consumption = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionTotal"]
estimated = df_power_breakdown_history['response_1']['history'][0]["isEstimated"]

# create a dictionary with first values for this zone
us_pnm1 = {'region':region,'datetime':datetime,'nuclear':nuclear,'geothermal':geothermal,'biomass':biomass, 'coal':coal, 'wind':wind, 'solar':solar, 
           'hydro':hydro, 'gas':gas, 'oil':oil, 'unknown':unknown, 'hydro-discharge':hydro_discharge, 
           'battery_discharge':battery_discharge, 'renewable_percentage':renewable_percentage, 'total_consumption':total_consumption, 
           'estimated':estimated}

# Create a dataFrame with the first values
df_US = pd.DataFrame.from_dict(us_pnm1,orient='index')


In [391]:
# Data wrangling from the response to create a legible dataFrame
# outer for loop for regions/responses
for reg in range(len(zones)):
    #for each zone
    response = f"response_{reg+1}"
   
# pull data from json for each time in this file for this region and add to the existing dataframe
    for i in range(24):
        # 24 is for the 24 hours of data for each zone
        region = df_power_breakdown_history[f"{response}"]['history'][i]['zone']
        datetime = df_power_breakdown_history[f"{response}"]['history'][i]['datetime']
        nuclear = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['nuclear']
        geothermal = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['geothermal']
        biomass = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['biomass']
        coal = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['coal']
        wind = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['wind']
        solar = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['solar']
        hydro = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['hydro']
        gas = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['gas']
        oil = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['oil']
        unknown = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['unknown']
        hydro_discharge = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['hydro discharge']
        battery_discharge = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['battery discharge']
        renewable_percentage = df_power_breakdown_history[f"{response}"]['history'][i]["renewablePercentage"]
        total_consumption = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionTotal"]
        estimated = df_power_breakdown_history[f"{response}"]['history'][i]["isEstimated"]

        # this 24 is also for the 23 hours of data for each zone
        df_US[24*reg+i]= {'region':region, 'datetime':datetime,'nuclear':nuclear,'geothermal':geothermal,'biomass':biomass, 'coal':coal, 'wind':wind, 'solar':solar, 
           'hydro':hydro, 'gas':gas, 'oil':oil, 'unknown':unknown, 'hydro-discharge':hydro_discharge, 
           'battery_discharge':battery_discharge, 'renewable_percentage':renewable_percentage, 'total_consumption':total_consumption, 
           'estimated':estimated}
#set up the times as rows and measurements as columns
df_US_new = df_US.transpose()

# check data types
df_US_new.describe()

C:\Users\mrkol\AppData\Local\Temp\ipykernel_28628\3249227210.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_US[24*reg+i]= {'region':region, 'datetime':datetime,'nuclear':nuclear,'geothermal':geothermal,'biomass':biomass, 'coal':coal, 'wind':wind, 'solar':solar,


,region,datetime,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption,estimated
count,1344,1344,1332,1332,1332,1332,1332,1332,1332,1332,1332,1332,1332,1332,1326,1332,1344
unique,53,24,305,24,54,516,417,380,499,843,39,271,4,26,91,1083,2
top,US-CAL-BANC,2024-08-22T21:00:00.000Z,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False
freq,48,56,847,1308,1218,621,721,726,384,206,1211,600,1329,1287,205,9,684


In [392]:
# fill NA values with zeroes for energy values
df_US_new = df_US_new.fillna({'nuclear': 0,'geothermal': 0,'biomass': 0, 'coal': 0, 'wind': 0, 'solar': 0, 
           'hydro': 0, 'gas': 0, 'oil': 0, 'unknown': 0, 'hydro-discharge':0, 'renewable_percentage':0,
           'battery_discharge':0, 'total_consumption':0})
df_US_new.describe()

,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption
count,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000
mean,1108.595238,12.908482,19.110863,1234.696429,948.262649,488.204613,480.782738,3178.309524,7.451637,82.179315,0.177827,18.598958,32.780506,7579.271577
std,2523.755009,96.054766,98.089754,3196.189036,3257.972012,2142.221041,1084.107099,6634.117005,115.248893,240.759706,3.848054,280.925334,34.708383,15338.536283
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,214.750000,0.000000,0.000000,0.000000,0.000000,4.000000,651.750000
50%,0.000000,0.000000,0.000000,18.500000,0.000000,0.000000,37.000000,834.000000,0.000000,4.000000,0.000000,0.000000,19.000000,2135.500000
75%,324.000000,0.000000,0.000000,1163.000000,238.250000,144.500000,488.000000,1910.000000,0.000000,66.250000,0.000000,0.000000,54.000000,6095.750000
max,14131.000000,773.000000,636.000000,27502.000000,22253.000000,19488.000000,8399.000000,44363.000000,2851.000000,4705.000000,103.000000,6177.000000,100.000000,91365.000000


In [393]:
# convert measured Energy values to integers in Giga Watts
#convert_dict = {'hydro': int}
convert_dict = {'nuclear': int, 'geothermal': int, 'biomass': int, 'coal': int, 'wind': int, 'solar': int, 'hydro': int, 'gas': int, 'oil': int, 
              'hydro-discharge': int, 'battery_discharge': int, 'renewable_percentage': int, 'total_consumption': int
               }
 # note - the unknown column only has values rarely - converting null values to integer doesn't work so this is left as an object
df_US_new = df_US_new.astype(convert_dict)

#check that data types are changed to int
df_US_new.dtypes

region                  object
datetime                object
nuclear                  int32
geothermal               int32
biomass                  int32
coal                     int32
wind                     int32
solar                    int32
hydro                    int32
gas                      int32
oil                      int32
unknown                  int64
hydro-discharge          int32
battery_discharge        int32
renewable_percentage     int32
total_consumption        int32
estimated               object
dtype: object

In [394]:
# Date Time work

# import datetime dependencies

from datetime import datetime

# set up lists to hold parsed data and DateTime as a datetime datetype
dates=[]
times = []
DateTime =[]

# convert date time strings
for i in range(len(df_US_new['datetime'])):

    # Parse the timestamp string to a datetime object
    dt_obj = datetime.strptime(df_US_new.iloc[i,1], '%Y-%m-%dT%H:%M:%S.%fZ')

    date = dt_obj.strftime('%Y-%m-%d')
    time = dt_obj.strftime('%H:%M:%S')

#add the new times and dates to lists

    dates.append(date)
    times.append(time)
    DateTime.append(dt_obj)

# add the times and dates to new columns in the data frame
df_US_new['UTC time'] = times
df_US_new['UTC date'] = dates
df_US_new['UTC DateTime'] = DateTime

In [395]:
#set the UTC DateTime as the index
df_US_new_reindex = df_US_new.set_index('UTC DateTime', inplace=True)

#drop the datetime column that contains a string
df_US_newer = df_US_new.drop('datetime', axis=1)
df_US_newer.head()

,region,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption,estimated,UTC time,UTC date
UTC DateTime,,,,,,,,,,,,,,,,,,
2024-08-22 21:00:00,US-SW-PNM,0,0,0,165,897,800,0,568,0,0,0,0,70,2430,False,21:00:00,2024-08-22
2024-08-22 22:00:00,US-SW-PNM,0,0,0,309,1009,390,0,624,0,0,0,0,60,2332,False,22:00:00,2024-08-22
2024-08-22 23:00:00,US-SW-PNM,0,0,0,307,1121,264,0,631,0,0,0,0,60,2323,False,23:00:00,2024-08-22
2024-08-23 00:00:00,US-SW-PNM,0,0,0,306,707,225,0,668,0,0,0,0,49,1906,False,00:00:00,2024-08-23
2024-08-23 01:00:00,US-SW-PNM,0,0,0,306,449,33,0,674,0,134,0,0,30,1596,False,01:00:00,2024-08-23


Section 3
Transform carbon intensity data

In [396]:
# pull data from C intensity json in dataframe
region = df_carbon_intensity_history['response_1']['history'][0]['zone']
datetime = df_carbon_intensity_history['response_1']['history'][0]['datetime']
carbon_Intensity = df_carbon_intensity_history['response_1']['history'][0]["carbonIntensity"]
estimated = df_carbon_intensity_history['response_1']['history'][0]["isEstimated"]

# create a dictionary with first values for this zone
us_pnm1C = {'region':region,'datetime':datetime,'Carbon_Intensity':carbon_Intensity, 'estimated':estimated}

# Create a dataFrame with the first values
df_US_C = pd.DataFrame.from_dict(us_pnm1C,orient='index')

In [397]:
# Data wrangling from the response to create a legible dataFrame for carbon intensity history

# outer for loop for regions/responses
for reg in range(len(zones)):
    response = f"response_{reg+1}"
    
# pull data from json for each time in this file for this region and add to the existing dataframe
    for i in range(24):
        # 24 is for the 24 hours of data for each zone
        region = df_carbon_intensity_history[f"{response}"]['history'][i]['zone']
        datetime = df_carbon_intensity_history[f"{response}"]['history'][i]['datetime']
        carbon_Intensity = df_carbon_intensity_history['response_1']['history'][i]["carbonIntensity"]
        estimated = df_carbon_intensity_history['response_1']['history'][i]["isEstimated"]
        
        # this 24 is also for the 24 hours of data in each zone
        df_US_C[24*reg+i]= {'region':region, 'datetime':datetime,'Carbon_Intensity':carbon_Intensity, 'estimated':estimated}

# make the datetime the rows and carbon_intensity a column
df_US_C_new = df_US_C.transpose()

# check data types
df_US_C_new.dtypes

C:\Users\mrkol\AppData\Local\Temp\ipykernel_28628\3332989139.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_US_C[24*reg+i]= {'region':region, 'datetime':datetime,'Carbon_Intensity':carbon_Intensity, 'estimated':estimated}


region              object
datetime            object
Carbon_Intensity    object
estimated           object
dtype: object

In [398]:
# convert carbon intensity measurement to an integer in g CO2e/kWh
convert_dict_C= {'Carbon_Intensity': int}
 
df_US_C_new = df_US_C_new.astype(convert_dict_C)

#check that the datatype has been changed
df_US_C_new.dtypes

region              object
datetime            object
Carbon_Intensity     int32
estimated           object
dtype: object

In [399]:
# add the times and dates to new columns in the data frame   -    This assumes the data for carbon intensity is pulled at the same time as power breakdown
df_US_C_new['UTC time'] = times
df_US_C_new['UTC date'] = dates
df_US_C_new['UTC DateTime'] = DateTime

#set the UTC DateTime as the index
df_US_C_new_reindex = df_US_C_new.set_index('UTC DateTime', inplace=True)
#drop the datetime column that contains a string
df_US_C_newer = df_US_C_new.drop('datetime', axis=1)

Section 4
Merge dataframes

In [400]:
df_power_and_carbon= pd.merge(df_US_newer, df_US_C_newer,on=['UTC DateTime','region','UTC time','UTC date'])



df_power_and_carbon.rename(columns={'Carbon_Intensity':'Carbon_Intensity(gCO2eq/kWh)','total_consumption':'total_consumption(GW)', 'nuclear':'nuclear(GW)', 
                                    'geothermal':'geothermal(GW)', 'biomass':'biomass(GW)', 'coal':'coal(GW)', 'wind':'wind(GW)', 'solar':'solar(GW)', 
                                    'hydro':'hydro(GW)','gas':'gas(GW)', 'region_x': 'region', 'estimated_x': 'breakdown estimated?','estimated_y':'intensity estimated?'}, inplace=True)




df_power_and_carbon.head()

,region,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption(GW),breakdown estimated?,UTC time,UTC date,Carbon_Intensity(gCO2eq/kWh),intensity estimated?
UTC DateTime,,,,,,,,,,,,,,,,,,,,
2024-08-22 21:00:00,US-SW-PNM,0,0,0,165,897,800,0,568,0,0,0,0,70,2430,False,21:00:00,2024-08-22,223,False
2024-08-22 22:00:00,US-SW-PNM,0,0,0,309,1009,390,0,624,0,0,0,0,60,2332,False,22:00:00,2024-08-22,318,False
2024-08-22 23:00:00,US-SW-PNM,0,0,0,307,1121,264,0,631,0,0,0,0,60,2323,False,23:00:00,2024-08-22,319,False
2024-08-23 00:00:00,US-SW-PNM,0,0,0,306,707,225,0,668,0,0,0,0,49,1906,False,00:00:00,2024-08-23,396,False
2024-08-23 01:00:00,US-SW-PNM,0,0,0,306,449,33,0,674,0,134,0,0,30,1596,False,01:00:00,2024-08-23,528,False


In [401]:
# import previous cleaned file into a pandas dataframe
df_us_energy = pd.read_csv('data/allRegions/runningUSenergy_data.csv')
df_us_energy_reindex=df_us_energy.set_index("UTC DateTime")
df_us_energy_reindex.describe()

C:\Users\mrkol\AppData\Local\Temp\ipykernel_28628\1527088667.py:2: DtypeWarning: Columns (16,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_us_energy = pd.read_csv('data/allRegions/runningUSenergy_data.csv')


,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption(GW),Carbon_Intensity(gCO2eq/kWh)
count,48567.000000,48567.000000,48567.000000,48567.000000,48567.000000,48567.000000,48567.000000,48567.000000,48577.000000,48577.000000,48577.000000,48577.000000,48479.000000,48539.000000,48567.000000
mean,1189.365351,14.154241,20.591183,1542.294645,704.549179,498.485865,577.368316,3955.404740,3.319802,96.363423,0.305227,19.082735,32.188267,8626.276149,436.640641
std,2550.895598,101.608983,101.703791,4078.031102,2509.283418,2082.042853,1284.090166,8018.916883,34.963369,292.150360,9.495042,285.316551,34.741055,16528.474853,187.449711
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,113.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,236.000000,0.000000,0.000000,0.000000,0.000000,4.000000,737.000000,300.000000
50%,0.000000,0.000000,0.000000,18.000000,0.000000,0.000000,67.000000,952.000000,0.000000,6.000000,0.000000,0.000000,18.000000,2221.000000,398.000000
75%,649.000000,0.000000,0.000000,1173.500000,192.000000,146.000000,605.000000,2693.000000,0.000000,74.000000,0.000000,0.000000,51.000000,8116.000000,537.000000
max,14865.000000,819.000000,2416.000000,49357.000000,25406.000000,20227.000000,65867.000000,79396.000000,2849.000000,13129.000000,557.000000,7475.000000,100.000000,128873.000000,1250.000000


In [402]:
df_us_energy_reindex.drop_duplicates(['UTC date', 'UTC time', 'region'],keep='last', inplace=True)
df_us_energy_reindex.describe()

,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption(GW),Carbon_Intensity(gCO2eq/kWh)
count,48567.000000,48567.000000,48567.000000,48567.000000,48567.000000,48567.000000,48567.000000,48567.000000,48577.000000,48577.000000,48577.000000,48577.000000,48479.000000,48539.000000,48567.000000
mean,1189.365351,14.154241,20.591183,1542.294645,704.549179,498.485865,577.368316,3955.404740,3.319802,96.363423,0.305227,19.082735,32.188267,8626.276149,436.640641
std,2550.895598,101.608983,101.703791,4078.031102,2509.283418,2082.042853,1284.090166,8018.916883,34.963369,292.150360,9.495042,285.316551,34.741055,16528.474853,187.449711
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,113.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,236.000000,0.000000,0.000000,0.000000,0.000000,4.000000,737.000000,300.000000
50%,0.000000,0.000000,0.000000,18.000000,0.000000,0.000000,67.000000,952.000000,0.000000,6.000000,0.000000,0.000000,18.000000,2221.000000,398.000000
75%,649.000000,0.000000,0.000000,1173.500000,192.000000,146.000000,605.000000,2693.000000,0.000000,74.000000,0.000000,0.000000,51.000000,8116.000000,537.000000
max,14865.000000,819.000000,2416.000000,49357.000000,25406.000000,20227.000000,65867.000000,79396.000000,2849.000000,13129.000000,557.000000,7475.000000,100.000000,128873.000000,1250.000000


In [403]:
# concatentate current data with existing file
df_both = pd.concat([df_us_energy_reindex,df_power_and_carbon])

# drop duplicate rows
df_both.drop_duplicates(['UTC date', 'UTC time', 'region'],keep='first', inplace=True)
df_both.describe()

,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption(GW),Carbon_Intensity(gCO2eq/kWh)
count,49044.000000,49044.000000,49044.000000,49044.000000,49044.000000,49044.000000,49044.000000,49044.000000,49054.000000,49054.000000,49054.000000,49054.000000,48956.00000,49016.000000,49044.000000
mean,1188.525752,14.140262,20.575973,1539.588941,705.753731,502.985380,576.128476,3948.955326,3.295695,96.346108,0.302259,18.927243,32.23325,8620.464461,434.234259
std,2550.765066,101.521022,101.623076,4070.729372,2512.506063,2092.593411,1282.019171,8007.891243,34.795471,292.451587,9.448811,283.960478,34.75474,16524.914381,188.152857
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,113.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,236.000000,0.000000,0.000000,0.000000,0.000000,4.00000,734.000000,296.000000
50%,0.000000,0.000000,0.000000,17.000000,0.000000,0.000000,66.000000,949.000000,0.000000,6.000000,0.000000,0.000000,18.00000,2219.000000,396.000000
75%,649.000000,0.000000,0.000000,1171.000000,192.000000,150.000000,605.000000,2676.000000,0.000000,74.000000,0.000000,0.000000,51.00000,8110.000000,530.000000
max,14865.000000,819.000000,2416.000000,49357.000000,25406.000000,20227.000000,65867.000000,79396.000000,2849.000000,13129.000000,557.000000,7475.000000,100.00000,128873.000000,1250.000000


In [404]:
df_both.to_csv(f'data/allRegions/runningUSenergy_data.csv')

In [405]:
df_both_cleaned = df_both.loc[df_both['breakdown estimated?']==False,:]
df_both_cleaned.describe()

,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption(GW),Carbon_Intensity(gCO2eq/kWh)
count,13782.000000,13782.000000,13782.000000,13782.000000,13782.000000,13782.000000,13782.000000,13782.000000,13782.000000,13782.000000,13782.0,13782.000000,13684.000000,13766.000000,13782.000000
mean,1461.802569,47.163256,67.769337,1705.117254,1239.815411,668.371136,935.224133,5037.843927,6.302641,102.706864,0.0,65.362284,32.195995,11350.721851,436.439922
std,2349.653849,181.187658,174.808141,4083.133208,3291.943601,2734.996352,1562.855758,7513.407184,48.505025,224.398228,0.0,528.814626,32.171570,16186.421441,169.822655
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,113.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000,253.250000,0.000000,0.000000,0.0,0.000000,5.000000,861.250000,320.000000
50%,2.000000,0.000000,0.000000,31.500000,35.000000,0.000000,190.000000,1447.000000,0.000000,4.000000,0.0,0.000000,21.000000,3315.500000,412.000000
75%,2254.000000,0.000000,2.000000,1053.000000,516.000000,99.000000,1228.000000,8309.000000,0.000000,91.750000,0.0,0.000000,49.000000,17332.000000,527.000000
max,14865.000000,818.000000,2416.000000,35359.000000,25406.000000,20227.000000,65867.000000,79396.000000,2054.000000,2818.000000,0.0,7475.000000,100.000000,117668.000000,1250.000000


In [406]:
df_both_cleaned.to_csv(f'data/allRegions/runningUSenergy_data_filtered.csv')